## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Importing Libraries</p>

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import torch
from mlxtend.classifier import StackingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from scipy.stats import mode
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import (
    roc_auc_score, roc_curve, auc, classification_report, 
    confusion_matrix, accuracy_score, matthews_corrcoef, f1_score,
    ConfusionMatrixDisplay
)
from sklearn.model_selection import (
    train_test_split, GridSearchCV, StratifiedShuffleSplit,
    StratifiedKFold, train_test_split, cross_val_score, cross_validate
)
import warnings

# Custom metric function
def custom_mcc_metric(y_true, y_pred):
    mcc = matthews_corrcoef(y_true, y_pred)
    return mcc

custom_mcc_scorer = make_scorer(name='mcc',
                                score_func=custom_mcc_metric,
                                greater_is_better=True)

warnings.filterwarnings('ignore')
print(torch.cuda.is_available())

  ### <p style="background-color: #fdefff;color:#c12eff;display: inline-block;padding:.6rem;border-radius:.5rem;border: 1px solid #c059ff">Loading data</p>

In [2]:
cd /workspace/data/

/workspace/data


In [3]:
train_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/train.csv"))
test_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/test.csv"))
sample_submission_data = pd.read_csv(os.path.join(os.getcwd(), "kaggle/playground-series-s4e8/sample_submission.csv"))

print("train_data :", train_data.shape)
print("test_data :", test_data.shape)
print("sample_submission_data :", sample_submission_data.shape)

train_data : (3116945, 22)
test_data : (2077964, 21)
sample_submission_data : (2077964, 2)


In [4]:
target = 'class'
features = train_data.drop(target, axis=1).columns.to_list()

categorical_features = train_data[features].select_dtypes(include='object').columns.to_list()

def cleaner(df):
    for col in categorical_features:
        df[col] = df[col].fillna('missing')
        df.loc[df[col].value_counts(dropna=False)[df[col]].values < 100, col] = "noise"
        df[col] = df[col].astype('category')

    return df
    
train_data = cleaner(train_data)
test_data = cleaner(test_data)

cap_diameter_mean = pd.concat([train_data['cap-diameter'], test_data['cap-diameter']]).mean(numeric_only=True)
train_data['cap-diameter'].fillna(cap_diameter_mean, inplace=True)
test_data['cap-diameter'].fillna(cap_diameter_mean, inplace=True)

X = train_data.copy()
y = X.pop(target)

lab_enc = LabelEncoder().fit(y)
y = lab_enc.transform(y)

def model_trainer(model, X, y, n_splits=5, random_state=42):
    skfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    oof_probs, oof_mccs = [], []
    print("="*80)
    print(f"Training {model.__class__.__name__}")
    print("="*80, end="\n")
    for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
        X_train, y_train = X.iloc[train_idx, :], y[train_idx]
        X_test, y_test = X.iloc[test_idx, :], y[test_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mcc = matthews_corrcoef(y_pred, y_test)
        oof_mccs.append(mcc)
        oof_probs.append(model.predict_proba(test_data))
        print(f"--- Fold {fold+1} MCC Score: {mcc:.6f}")
    print(f"\n---> Mean MCC Score: {np.mean(oof_mccs):.6f} \xb1 {np.std(oof_mccs):.6f}\n\n")
    return oof_probs, oof_mccs

xgb_params = {
    'n_estimators': 2407,
    'eta': 0.009462133032592785,
    'gamma': 0.2865859948765318,
    'max_depth': 31,
    'min_child_weight': 47,
    'subsample': 0.6956431754146083,
    'colsample_bytree': 0.3670732604094118,
    'grow_policy': 'lossguide',
    'max_leaves': 73,
    'enable_categorical': True,
    'n_jobs': -1,
    'device': 'cuda',
    'tree_method': 'hist'
} # 0.9844272567086021

cat_params = {
    'iterations': 1041,
    'learning_rate': 0.08777255350163136,
    'depth': 10,
    'l2_leaf_reg': 0.1259643500248322,
    'bootstrap_type': 'Bayesian',
    'random_strength': 4.276181166674371e-08,
    'bagging_temperature': 0.35995482350907326,
    'od_type': 'Iter',
    'od_wait': 39,
    "verbose": False,
    "allow_writing_files": False,
    "task_type": 'GPU',
    "cat_features": categorical_features
} # 0.9841773055825763

lgb_params = {
    'n_estimators': 2500,
    'random_state':42,
    'max_bin':1024,
    'colsample_bytree':0.6,
    'reg_lambda': 80,
#     'device': 'gpu',
    'verbosity': -1
}

oof_probs = {}
oof_probs['xgb'], _ = model_trainer(XGBClassifier(**xgb_params), X, y, random_state=101)
oof_probs['cat'], _ = model_trainer(CatBoostClassifier(**cat_params), X, y, random_state=101)
oof_probs['lgb'], _ = model_trainer(LGBMClassifier(**lgb_params), X, y, random_state=101)

oof_preds = {}
for model in oof_probs.keys():
    oof_preds[model] = np.argmax(np.mean(oof_probs[model], axis=0), axis=1)



Training XGBClassifier
--- Fold 1 MCC Score: 0.984589
--- Fold 2 MCC Score: 0.984413
--- Fold 3 MCC Score: 0.984830
--- Fold 4 MCC Score: 0.984426
--- Fold 5 MCC Score: 0.984419

---> Mean MCC Score: 0.984535 ± 0.000161


Training CatBoostClassifier
--- Fold 1 MCC Score: 0.984549
--- Fold 2 MCC Score: 0.984270
--- Fold 3 MCC Score: 0.984758
--- Fold 4 MCC Score: 0.984263
--- Fold 5 MCC Score: 0.984131

---> Mean MCC Score: 0.984394 ± 0.000228


Training LGBMClassifier
--- Fold 1 MCC Score: 0.984666
--- Fold 2 MCC Score: 0.984513
--- Fold 3 MCC Score: 0.984869
--- Fold 4 MCC Score: 0.984497
--- Fold 5 MCC Score: 0.984395

---> Mean MCC Score: 0.984588 ± 0.000165




FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/playground-series-s4e8/sample_submission.csv'

In [6]:
sub = pd.read_csv("./kaggle/playground-series-s4e8/sample_submission.csv")
preds = [pred for model, pred in oof_preds.items()]
md = mode(preds, axis=0)[0] if len(preds)>1 else preds[0]
sub[target] = lab_enc.inverse_transform(md)
sub.to_csv("submission111.csv", index=False)

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Handling NaN Values And Less Frequent Categories</p> 

In [ ]:
# Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
#        'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
#        'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
#        'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
#        'habitat', 'season', 'veil-info'],
#       dtype='object')

# 결측치와 특성 존재 여부를 동시에 다룰 수 있음: 앞에서 0과 1로 veil-type의 존재 여부를 표현하고, 뒤에서 'unknown'을 통해 veil-color의 결측치를 처리함으로써 두 정보를 효과적으로 결합할 수 있습니다.
# 모델 학습에 유용한 정보 제공: 결측치나 특성의 존재 여부를 단순히 무시하지 않고, 이들을 결합하여 모델이 더 많은 패턴을 학습할 수 있게 도와줍니다.
# Index(['w', 'y', 'n', 'u', 'k', 'e', 'g', 'p', 'r', 'o', 's', 'a', 't', 'd',
#        'i', 'h', 'c', 'f', 'l', 'b', 'z', '8.25', '2.49', '3.32'],
#       dtype='object', name='veil-color')
# Index(['u', 'w', 'a', 'f', 'e', 'b', 'c', 'y', 'k', 'g', 'n', 's', 'r', 'd',
#        'p', 'h', 'i', 'l', 'is None', 't', '21.11', '5.94'],
#       dtype='object', name='veil-type')

# Index(['f10', 'tnone', 't1', 't7', 't10', 't4', 't2', 't5', 't3', 't9',
#        'fnone', 'f1', 'f2', 'f7', 'f4', 't8', 'f5', 'r2', 't6', 'f3', 'l4',
#        't0', 'p5', 'z7', 'c10', 'x10', 'f9', 'f6', 's10', 'm9', 'hnone', 's1',
#        'g3', 'g5', 'h7', 'e1', 'f0', 'r4', 'dnone', 's5', 'cnone', 'h1', 'p1',
#        'h5', 'h10', 'w3', 'y2', 'a10', 'ynone', 'e10', 'p7', '10.310', 's2',
#        'o2', 'g10', 'h2', 'g1', 's3', 'p3', 'knone', 'inone', 'nnone', 'rnone',
#        'l5', 'c1', 'n10', 'c3', 'o10', 'e4', 'd10', 'f has-ring10', 'lnone',
#        'c7', 'e3', 'y1', 'k10'],
#       dtype='object', name='has-ring-type')

train_data['veil-info'] = train_data['veil-type'].notna().astype(int).astype(str) + train_data['veil-color'].fillna('unknown')
cap_shape_mapping = {'b': 0, 'c': 1, 'x': 2, 'f': 3, 's': 4, 'p': 5, 'o': 6}
cap_color_mapping = {'n': 0, 'b': 1, 'g': 2, 'r': 3, 'p': 4, 'u': 5, 'e': 6, 'w': 7, 'y': 8, 'l': 9, 'o': 10, 'k': 11}
ring_type_mapping = {'c': 0, 'e': 1, 'r': 2, 'g': 3, 'l': 4, 'p': 5, 's': 6, 'z': 7, 'y': 8, 'm': 9, 'f': 10}
train_data['cap-shape'] = train_data['cap-shape'].map(cap_shape_mapping).astype(str).str.replace('.0', '', regex=False)
train_data['cap-color'] = train_data['cap-color'].map(cap_color_mapping).astype(str).str.replace('.0', '', regex=False)
train_data['has-ring-type'] = (
    train_data['has-ring'] + 
    train_data['ring-type'].fillna('none').map(ring_type_mapping).apply(lambda x: f"{x:.0f}" if pd.notna(x) else 'none').str.replace('.0', '', regex=False)
)

test_data['veil-info'] = test_data['veil-type'].notna().astype(int).astype(str) + test_data['veil-color'].fillna('unknown')
test_data['cap-shape'] = test_data['cap-shape'].map(cap_shape_mapping).astype(str).str.replace('.0', '', regex=False)
test_data['cap-color'] = test_data['cap-color'].map(cap_color_mapping).astype(str).str.replace('.0', '', regex=False)
test_data['has-ring-type'] = (
    test_data['has-ring'] + 
    test_data['ring-type'].fillna('none').map(ring_type_mapping).apply(lambda x: f"{x:.0f}" if pd.notna(x) else 'none').str.replace('.0', '', regex=False)
)

train_data = train_data.drop(['id', 'veil-color', 'veil-type', 'has-ring', 'ring-type'], axis=1)
test_data = test_data.drop(['id', 'veil-color', 'veil-type', 'has-ring', 'ring-type'], axis=1)

In [ ]:
cat_feats = ['cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-root', 'stem-surface', 'stem-color', 'spore-print-color',
       'habitat', 'season', 'veil-info', 'has-ring-type']

for feat in cat_feats:
    train_data[feat] = train_data[feat].astype('category')
for feat in cat_feats:
    test_data[feat] = test_data[feat].astype('category')

def cleaning(df):
    threshold = 100
    for feat in cat_feats:
        if df[feat].dtype.name == 'category':
            # Add 'missing' and 'noise' to categories if not present
            if 'missing' not in df[feat].cat.categories:
                df[feat] = df[feat].cat.add_categories('missing')
            if 'noise' not in df[feat].cat.categories:
                df[feat] = df[feat].cat.add_categories('noise')
        else:
            # Convert to category and add new categories
            df[feat] = df[feat].astype('category')
            df[feat] = df[feat].cat.add_categories(['missing', 'noise'])
        
        # Fill missing values with 'missing'
        df[feat] = df[feat].fillna('missing')
        
        # Replace infrequent categories with 'noise'
        counts = df[feat].value_counts(dropna=False)
        infrequent_categories = counts[counts < threshold].index
        df[feat] = df[feat].apply(lambda x: 'missing' if x in infrequent_categories else x)
    
    return df

#
train_data = cleaning(train_data)
test_data = cleaning(test_data)

In [ ]:
# Calculate group means for the train data
group_by_features = ['stem-width', 'stem-height']
group_means_train = train_data.groupby(group_by_features)['cap-diameter'].mean()

def fill_na_with_group_mean(row, group_means):
    if pd.isna(row['cap-diameter']):
        group = tuple(row[group_by_features])
        return group_means.get(group, np.nan)
    else:
        return row['cap-diameter']

train_data['cap-diameter'] = train_data.apply(fill_na_with_group_mean, axis=1, group_means=group_means_train)
test_data['cap-diameter'] = test_data.apply(fill_na_with_group_mean, axis=1, group_means=group_means_train)

# Calculate the mode from the training data
cap_diameter_mode = train_data['cap-diameter'].mode()[0]
stem_height_mode = train_data['stem-height'].mode()[0]

# Fill missing values in the training data
train_data['cap-diameter'] = train_data['cap-diameter'].fillna(cap_diameter_mode)
test_data['cap-diameter'] = test_data['cap-diameter'].fillna(cap_diameter_mode)

train_data['stem-height'] = train_data['stem-height'].fillna(stem_height_mode)
test_data['stem-height'] = test_data['stem-height'].fillna(stem_height_mode)

## <p style="background-color:#d8ecff; color: #009dff;margin:0; display:inline-block;padding:.4rem;border-radius:.25rem;border:1px solid #009dff">Splitting Data</p> 

In [ ]:
X = train_data.drop(['class'], axis=1)
y = train_data['class']
print(X.shape, y.shape)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
print(y[:3])

# 모델링

## Using XGBoost with optuna

In [ ]:
# Callback to print the MCC score for each trial
def print_mcc_callback(study, trial):
    mcc = trial.user_attrs["mcc"]
    print(f"Trial {trial.number}: MCC = {mcc}")
    
def model_report(estimator, X, y, cv=5):
    print("="*80)
    print(f"    Model: {estimator.__class__.__name__}")
    print("="*80)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=1/cv, shuffle=True, stratify=y, random_state=42)
    
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    print(f"F1 Score : {f1.mean():.6f}")
    print(f"MCC Score: {mcc.mean():.6f}")
    
    ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred)).plot()
    plt.title("Confusion Matrix")
    plt.show()

    print()

xgb_clf = XGBClassifier(enable_categorical=True, device="cuda", tree_method="hist")
cat_clf = CatBoostClassifier(
    cat_features=cat_feats,
    verbose=False,
    allow_writing_files=False,
    task_type="GPU"
)
lgb_clf = LGBMClassifier(device='cpu', verbosity=-1)

## 기본값으로 모델링 테스트

In [ ]:
model_report(xgb_clf, X, y)
model_report(cat_clf, X, y)
model_report(lgb_clf, X, y)

## XGBClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4885)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1200), #  100, 1000
        'max_depth': trial.suggest_int('max_depth', 10, 22), #5, 10
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0), # 0.5, 1.0
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 10.0),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1.0, 10.0),
        "eta": trial.suggest_float("eta", 1e-3, 1e-2),
        "min_child_weight": trial.suggest_int("min_child_weight", 40, 100),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "max_leaves": trial.suggest_int("max_leaves", 40, 100), # 16, 84
        'device': 'cuda',
        'tree_method': 'gpu_hist'
    }

    model = XGBClassifier(**params, use_label_encoder=False, eval_metric='logloss', enable_categorical=True, early_stopping_rounds=20)
    # Fit the model with early stopping
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set
        verbose=False                 # Suppress output
    )
    
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc



storage = "sqlite:///xgb.db"

# Optimize hyperparameters with Optuna
study = optuna.create_study(storage=storage,
                            direction='maximize',
                           )
study.optimize(objective, n_trials=100, callbacks=[print_mcc_callback])

# Get the best parameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

## CatBoostClassifier

In [ ]:
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        "random_strength": trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        "od_wait": trial.suggest_int("od_wait", 10, 50),
        "verbose": False,
        "allow_writing_files": False,
        "task_type": 'GPU',
        "cat_features": cat_feats
    }

    model = CatBoostClassifier(**params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set
        early_stopping_rounds=20,
        verbose=False                 # Suppress output
    )
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc


study_name = "cat"
storage = "sqlite:///cat.db"

study = optuna.create_study(storage=storage,
                            study_name=study_name,
                            direction="maximize",
                            sampler=TPESampler(n_startup_trials=20, multivariate=True),
                            load_if_exists=True)

study.optimize(objective, n_trials=50, callbacks=[print_mcc_callback])

print(study.best_params)

## LGBMClassifier

In [ ]:
def objective(trial):
    params = {
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "device": 'cpu',
        "verbosity": -1
    }

    model = LGBMClassifier(**params, early_stopping_rounds=20)

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],  # Validation set
        # verbose=False                 # Suppress output
    )
    y_pred = model.predict(X_test)
    mcc = matthews_corrcoef(y_test, y_pred)
    trial.set_user_attr("mcc", mcc)
    return mcc


study_name = "lgb"
storage = "sqlite:///lgb.db"

study = optuna.create_study(storage=storage,
                            study_name=study_name,
                            direction="maximize",
                            sampler=TPESampler(n_startup_trials=20, multivariate=True),
                            load_if_exists=True)

study.optimize(objective, n_trials=100, callbacks=[print_mcc_callback])

print(study.best_params)

In [ ]:
# parameters={'n_estimators': 297, 'max_depth': 16, 'learning_rate': 0.03906159386409017, 'subsample': 0.6935900010487451, 'colsample_bytree': 0.5171160704967471, 'gamma': 0.00013710778966124443, 'lambda': 0.0017203271581656767, 'alpha': 8.501510750413265e-06, 'scale_pos_weight': 1.0017942891559255,'enable_categorical': True,'tree_method': 'hist', 'device': 'cuda'}

# 0.984876834594928
xgb_parameters={'n_estimators': 629, 'max_depth': 16, 'learning_rate': 0.10540811331505591, 'subsample': 0.9980081491289166, 'colsample_bytree': 0.7065318052513923, 'gamma': 2.8307622020907574e-06, 'lambda': 0.02456026309353021, 'alpha': 1.5584362263554227e-07, 'scale_pos_weight': 1.105994872793953, 'eta': 0.0010400771543147627, 'min_child_weight': 79, 'grow_policy': 'depthwise', 'max_leaves': 61,
               'device': 'cuda'
               }
# 0.98473160011893
cat_parameters={'iterations': 1472, 'learning_rate': 0.08436268683474936, 'depth': 8, 'l2_leaf_reg': 0.2621416701020154, 'bootstrap_type': 'Bayesian', 'random_strength': 4.490114031912514e-07, 'bagging_temperature': 0.42447506181344297, 'od_type': 'Iter', 'od_wait': 21,
                "task_type": 'GPU',
                "cat_features": cat_feats}
# 0.9848032057376603
lgb_parameters={'lambda_l1': 1.3323938321586053e-07, 'lambda_l2': 0.07774032281247541, 'num_leaves': 248, 'feature_fraction': 0.41087472855265783, 'bagging_fraction': 0.9928696340850532, 'bagging_freq': 4, 'min_child_samples': 97} 

In [ ]:
def model_trainer(model, X, y, n_splits=5, random_state=4885):
    skfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    oof_probs, oof_mccs = [], []
    print("="*80)
    print(f"Training {model.__class__.__name__}")
    print("="*80, end="\n")
    for fold, (train_idx, test_idx) in enumerate(skfold.split(X, y)):
        X_train, y_train = X.iloc[train_idx, :], y[train_idx]
        X_test, y_test = X.iloc[test_idx, :], y[test_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mcc = matthews_corrcoef(y_pred, y_test)
        oof_mccs.append(mcc)
        oof_probs.append(model.predict_proba(test_data))
        print(f"--- Fold {fold+1} MCC Score: {mcc:.6f}")
    print(f"\n---> Mean MCC Score: {np.mean(oof_mccs):.6f} \xb1 {np.std(oof_mccs):.6f}\n\n")
    return oof_probs, oof_mccs

In [ ]:
oof_probs = {}
oof_probs['xgb'], _ = model_trainer(XGBClassifier(**xgb_parameters, enable_categorical=True), X, y, random_state=4885)
oof_probs['cat'], _ = model_trainer(CatBoostClassifier(**cat_parameters), X, y, random_state=4885)
oof_probs['lgb'], _ = model_trainer(LGBMClassifier(**lgb_parameters), X, y, random_state=4885)

In [ ]:
xgb_probs = np.mean(oof_probs['xgb'], axis=0)[:, 1]
cat_probs = np.mean(oof_probs['cat'], axis=0)[:, 1]
lgb_probs = np.mean(oof_probs['lgb'], axis=0)[:, 1]
average_probs = np.mean([xgb_probs, cat_probs, lgb_probs], axis=0)

In [ ]:
# _0.98508
# Average the probabilities for class 1 from the three models
average_probs = np.mean([xgb_probs, cat_probs, lgb_probs], axis=0)

# Assuming you have the 'id' column from the test data
id_column = sample_submission_data.pop('id')

# Create a DataFrame for the submission
submission_df = pd.DataFrame({
    'id': id_column,
    'class': average_probs
})
# Save the submission DataFrame to a CSV file
submission_df.to_csv('./kaggle/submission_xgb_cat_lgbm_0830_0.98508_prob.csv', index=False)
print("Submission file created: submission_autog_ensemble_avg.csv")

# 예측값으로 저장하는법

In [ ]:
oof_preds = {}
for model in oof_probs.keys():
    oof_preds[model] = np.argmax(np.mean(oof_probs[model], axis=0), axis=1)

In [ ]:
sub = pd.read_csv("./kaggle/playground-series-s4e8/sample_submission.csv")
preds = oof_preds['lgb']
sub['class'] = label_encoder.inverse_transform(preds)
sub.to_csv('./kaggle/playground-series-s4e8/submission_lgb_0830_111.csv', index=False)

In [ ]:
# sub = pd.read_csv("./kaggle/playground-series-s4e8/sample_submission.csv")
# preds = [pred for model, pred in oof_preds.items()]
# md = mode(preds, axis=0)[0] if len(preds)>1 else preds[0]
# sub['class'] = label_encoder.inverse_transform(md)
# sub.to_csv('./kaggle/playground-series-s4e8/submission_xgb_cat_lgbm_0830_111.csv', index=False)